In [ ]:
%matplotlib inline
%load_ext autoreload
%autoreload 2
import sys
sys.path.insert(0,'../')
sys.path.insert(0,'../../../data_analysis/')
from simulate_door_slam import *
from trajectory import *
from utils import *
from local_plot import *


In [ ]:
path = "/home/xuhao/data/d2slam/quadcam_7inch_n3_2023_1_14/outputs/doorslam-5-yaw"
nodes = [1, 2, 3, 4, 5]
thres = 0.01 #Terminate thresold for DGS
evaluate_door_slam(path, nodes, thres, step=10)

# paths_pgo, t0 = read_paths(path + "/door-slam/", nodes, prefix="pgo_")
# paths_rt, _ = read_paths(path + "/door-slam/", nodes, prefix="realtime_", t0=t0)
# paths_gt, _ = read_paths(path + "/../../", nodes, prefix="eight_yaw_", suffix="-groundtruth.txt", t0=t0)
# align_paths(paths_rt, paths_gt, align_by_first=True, align_with_minize=True)
# align_paths(paths_pgo, paths_gt, align_by_first=True, align_with_minize=True)
# dte=550
# display(plot_fused_err(nodes, paths_rt, paths_gt, poses_pgo=paths_pgo, dte=dte, show=False))
# display(relative_pose_err(nodes, paths_rt, paths_gt, dte=dte))

In [ ]:
path = "/home/xuhao/data/d2slam/tum_datasets//outputs/door-slam"
nodes = [1, 2, 3, 4, 5]
thres = 0.01 #Terminate thresold for DGS
evaluate_door_slam(path, nodes, thres, step=10)

# paths_pgo, t0 = read_paths(path + "/door-slam/", nodes, prefix="pgo_")
# paths_rt, _ = read_paths(path + "/door-slam/", nodes, prefix="realtime_", t0=t0)
# paths_gt, _ = read_paths(path + "/../../", nodes, prefix="groundtruth_", t0=t0)
# align_paths(paths_rt, paths_gt, align_by_first=True, align_with_minize=True)
# align_paths(paths_pgo, paths_gt, align_by_first=True, align_with_minize=True)
# dte=550
# display(plot_fused_err(nodes, paths_rt, paths_gt, poses_pgo=paths_pgo, dte=dte, show=False))
# display(relative_pose_err(nodes, paths_rt, paths_gt, dte=dte))

In [ ]:
path = "/home/xuhao/data/d2slam/tum_corr/outputs/door-slam"
nodes = [1, 2, 3, 4, 5]
thres = 0.01 #Terminate thresold for DGS
rt_paths = evaluate_door_slam(path, nodes, thres)
paths_pgo, t0 = read_paths(path + "/door-slam/", nodes, prefix="pgo_")
paths_rt, _ = read_paths(path + "/door-slam/", nodes, prefix="realtime_", t0=t0)
paths_gt, _ = read_paths(path + "/../../", nodes, prefix="groundtruth_", t0=t0)
align_paths(paths_rt, paths_gt, align_by_first=True, align_with_minize=True)
align_paths(paths_pgo, paths_gt, align_by_first=True, align_with_minize=True)
dte=550
display(plot_fused_err(nodes, paths_rt, paths_gt, poses_pgo=paths_pgo, dte=dte, show=True))
display(relative_pose_err(nodes, paths_rt, paths_gt, dte=dte))
plot_fused(nodes, paths_rt, paths_gt, poses_pgo=paths_pgo, figsize=(15,10))


In [ ]:
path = "/home/xuhao/data/d2slam/tum_corr/outputs/d2slam"
nodes = [1, 2, 3, 4, 5]
rt_paths = evaluate_door_slam(path, nodes, 0.01, step=20)
paths_pgo, t0 = read_paths(path + "/door-slam/", nodes, prefix="pgo_")
paths_rt, _ = read_paths(path + "/door-slam/", nodes, prefix="realtime_", t0=t0)
paths_gt, _ = read_paths(path + "/../../", nodes, prefix="groundtruth_", t0=t0)
align_paths(paths_rt, paths_gt, align_by_first=True, align_with_minize=True)
align_paths(paths_pgo, paths_gt, align_by_first=True, align_with_minize=True)
dte=550
# display(plot_fused_err(nodes, paths_rt, paths_gt, poses_pgo=paths_pgo, dte=dte, show=True))
display(relative_pose_err(nodes, paths_rt, paths_gt, dte=dte))
plot_fused(nodes, paths_rt, paths_gt, poses_pgo=paths_pgo, figsize=(15,10))

In [ ]:
pgo = PoseGraph()
pgo.read_g2o_folder("/home/xuhao/data/d2slam/tum_corr/outputs/d2slam/door-slam/rot_initalized/")
frame_id_to_stamp = {}
for i in nodes:
    with open(f"/home/xuhao/data/d2slam/tum_corr/outputs/d2slam/swarm{i}/frame_timestamp.txt", "r") as f:
        for line in f:
            frame_id, stamp = line.split()
        frame_id_to_stamp[int(frame_id)] = float(stamp)
pgo_latest_states = {i:[] for i in nodes}
for agent_id in pgo.agents:
    # Find the keyframe with largest frame_id
    agent = pgo.agents[agent_id]
    robot_idx = agent_id + 1
    keyframe_ids = agent.get_keyframe_ids()
    for frame_id in keyframe_ids:
        ts = frame_id_to_stamp[frame_id]
        frame = pgo.keyframes[frame_id]
        pos, quat = frame.pos, frame.quat
        # Inverse transform by the first keyframe
        pgo_latest_states[robot_idx].append(np.concatenate(([ts], pos, quat)))
    pgo_latest_states[robot_idx] = np.array(pgo_latest_states[robot_idx])
np.savetxt(f"/home/xuhao/data/d2slam/tum_corr/outputs/d2slam/door-slam/rot_initalized/pgo_{i}.csv", pgo_latest_states[i], delimiter=" ")
